In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [3]:
base_path = "C:/Users/Dell/Downloads/GSASLdatasets/train/"
categories = {0: "A",
                1: "B",
                2: "C",
                3: "D",
                4: "E",
                5: "F",
                6: "G",
                7: "H",
                8: "I",
                9: "J",
                10: "K",
                11: "L",
                12: "M",
                13: "N",
                14: "O",
                15: "P",
                16: "Q",
                17: "R",
                18: "S",
                19: "T",
                20: "U",
                21: "V",
                22: "W",
                23: "X",
                24: "Y",
                25: "Z",
                26: "del",
                27: "nothing",
                28: "space",
   
}

In [4]:
# Function to preprocess images
def preprocess_image(path, image_size):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (image_size, image_size))
    img = img.astype('float32') / 255.0  # Normalization
    return img

In [8]:
# Set the paths to your dataset folders
train_path = "C:/Users/Dell/Downloads/GSASLdatasets/train/"
val_path = "C:/Users/Dell/Downloads/GSASLdatasets/val/"
test_path = "C:/Users/Dell/Downloads/GSASLdatasets/test/"

# Function to load images from a folder
def load_images_from_folder(folder):
    data = []
    labels = []
    categories = os.listdir(folder)
    for category in categories:
        label = categories.index(category)  # Assigning labels based on folder names
        category_path = os.path.join(folder, category)
        for filename in os.listdir(category_path):
            img_path = os.path.join(category_path, filename)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (100, 100))
            image = image.astype('float32') / 255.0  # Normalization
            data.append(image)
            labels.append(label)
    return np.array(data), np.array(labels)

# Load train, validation, and test datasets
train_data, train_labels = load_images_from_folder(train_path)
val_data, val_labels = load_images_from_folder(val_path)
test_data, test_labels = load_images_from_folder(test_path)



In [108]:
class Conv2D:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / (filter_size * filter_size)
        self.bias = np.zeros((num_filters, 1))
    
    def iterate_regions(self, image):
        if len(image.shape) == 3:
            h, w, _ = image.shape  # Get height, width, and channels
        elif len(image.shape) == 2:
            h, w = image.shape  # Assume single-channel image
        else:
            raise ValueError("Unexpected input shape")

        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                if len(image.shape) == 3:
                    im_region = image[i:(i + self.filter_size), j:(j + self.filter_size), :]
                else:
                    im_region = image[i:(i + self.filter_size), j:(j + self.filter_size)]
                yield im_region, i, j


    def forward(self, input):
        self.last_input = input
        if len(input.shape) == 2:
            input = np.expand_dims(input, axis=-1)  # Add the channel dimension
            h, w, _ = input.shape
        elif len(input.shape) == 3:
            h, w, _ = input.shape
        else:
            raise ValueError("Unexpected input shape")

        output_h = h - self.filter_size + 1
        output_w = w - self.filter_size + 1
        output = np.zeros((output_h, output_w, self.num_filters))

        for im_region, i, j in self.iterate_regions(input):
            for f in range(self.num_filters):
                # Ensure compatibility by expanding the dimensions
                expanded_filter = np.expand_dims(self.filters[f, :, :], axis=-1)
                # Perform element-wise multiplication and sum along the channels
                output[i, j, f] = np.sum(im_region * expanded_filter) + self.bias[f]

        return output

In [109]:
class MaxPool2D:
    def __init__(self, filter_size):
        self.filter_size = filter_size

    def iterate_regions(self, image):
        h, w, num_filters = image.shape
        new_h = h // self.filter_size
        new_w = w // self.filter_size

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * self.filter_size):(i * self.filter_size + self.filter_size),
                                  (j * self.filter_size):(j * self.filter_size + self.filter_size), :]
                yield im_region, i, j

    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output_h = h // self.filter_size
        output_w = w // self.filter_size
        output = np.zeros((output_h, output_w, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            for f in range(num_filters):
                output[i, j, f] = np.amax(im_region[:, :, f])

        return output


In [155]:
learning_rate = 0.01
class Dense:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2.0 / input_size)
        self.biases = np.zeros((1, output_size))

    def forward(self, input):
        self.last_input = input
        if len(input.shape) != 1:
            input = input.reshape(-1, 1)  # Reshape the input to match the expected shape
        output = np.dot(input.T, self.weights) + self.biases  # Perform dot product
        return output
    
    def backward(self, gradients):
        # Calculate gradients with respect to the layer's input
        input_gradients = np.dot(gradients, self.weights.T)

        # Update weights and biases using gradients (for learning purposes)
        weights_gradients = np.dot(self.last_input.T, gradients)
        biases_gradients = gradients.mean(axis=0, keepdims=True)

        # Update weights and biases
        self.weights -= learning_rate * weights_gradients
        self.biases -= learning_rate * biases_gradients

        return input_gradients


In [170]:
class ReLU:
    def forward(self, input):
        self.input = input
        return np.maximum(0, input)
    
    def backward(self, gradients):
        return gradients.reshape(self.input.shape) * (self.input > 0).astype(float)


In [171]:
# Define your model architecture
conv1 = Conv2D(num_filters=64, filter_size=3)
pool1 = MaxPool2D(filter_size=2)
conv2 = Conv2D(num_filters=128, filter_size=3)
pool2 = MaxPool2D(filter_size=2)
dense1 = Dense(67712, 512)  
dense2 = Dense(512, 29)  # Assuming num_classes is defined
relu = ReLU()

# Define forward pass
def forward(image, label):
    out = conv1.forward(image)
    out = relu.forward(out)
    out = pool1.forward(out)
    
    out = conv2.forward(out)
    out = relu.forward(out)
    out = pool2.forward(out)
    
    # Flatten the output from the convolutional layers
    flattened = out.reshape(-1)
    
    out = dense1.forward(flattened)
    out = relu.forward(out)
    out = dense2.forward(out)
    
    # Calculate loss
    loss = (out - label) ** 2
    
    return out, loss



In [174]:
epochs = 15
learning_rate = 0.01

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Training
    for i, image in enumerate(train_data):
        label = train_labels[i]
        
        # Forward pass
        out, loss = forward(image, label)

        # Backward pass
        dense2_gradients = 2 * (out - label)
        dense2_input_gradients = dense2.backward(dense2_gradients)
        
        dense1_gradients = dense2.weights.T.dot(dense2_gradients.T).T
        dense1_input_gradients = relu.backward(dense1.last_input) * dense1_gradients
        
        dense2.weights -= learning_rate * np.outer(dense1.last_input, dense2_gradients)
        dense1.weights -= learning_rate * np.outer(flattened, dense1_input_gradients)
        
        dense1_input_gradients = dense1.backward(dense1.last_input)
        
        pool2_input_gradients = dense1_input_gradients.reshape(pool2_out.shape)
        conv2_input_gradients = pool2.backward(pool2_input_gradients)
        
        pool1_input_gradients = conv2_input_gradients
        conv1_input_gradients = pool1.backward(pool1_input_gradients)
        
        # Update weights for convolutional layers
        conv2.filters -= learning_rate * conv2_input_gradients
        conv1.filters -= learning_rate * conv1_input_gradients


Epoch 1/15


ValueError: shapes (29,512) and (29,1) not aligned: 512 (dim 1) != 29 (dim 0)